/*
 * Copyright 2021 ConsenSys Software Inc.
 *
 * Licensed under the Apache License, Version 2.0 (the "License"); you may 
 * not use this file except in compliance with the License. You may obtain 
 * a copy of the License at http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software dis-
 * tributed under the License is distributed on an "AS IS" BASIS, WITHOUT 
 * WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the 
 * License for the specific language governing permissions and limitations 
 * under the License.
 */

# Notebook description

This notebook contains some basic processing to automate the collection of statistics relating to the Dafny files.
By creating functions to perform analysis of Dafny files, additional results can easily be added to the pandas dataframe.
The use of a pandas dataframe provides many options for visualisation and the data can easily by stored in a csv.
The data can also easily be supplemented with timestamps to faciliate time series analysis.

This file is a working file and will be converted to a python script in due course.

# TODO: Reformat function documentation to standard style used within this repo

## Libraries

In [20]:
# import libraries
import os
import subprocess

import pandas as pd
import re
import numpy as np

import time
import shutil

## File processing functions

In [21]:
# find *.dfy files, within a given local repo path
# this function will search all subfolders of dirName
# a sorted list of files is returned
def getListOfDafnyFiles(dirName,exclude_folders=[]):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        # if entry is a directory then append the list of files in this directory to allFiles
        if os.path.isdir(fullPath):
            if os.path.abspath(fullPath) not in exclude_folders:
                allFiles = allFiles + getListOfDafnyFiles(fullPath, exclude_folders)
        # else append file only if it is a Dafny file
        else:
            if entry.endswith(".dfy"):
                allFiles.append(fullPath)
    return sorted(allFiles)   

In [22]:
# find folders within the repo that have *.dfy files
# a sorted list of folders is returned (i.e. full path of each folder)
def getListOfDafnyFolders(dafnyFiles):
    listOfDirectories = list()
    for file in dafnyFiles:
        listOfDirectories.append(os.path.dirname(file))
    return sorted(list(set(listOfDirectories)))

In [23]:
# get folder for an individual dafny file
# i.e. for the full path of a dafny file, the filename and repo path are striped
def getFolder(repo, dafny_file):
    repo_path, folder = os.path.dirname(dafny_file).split(repo,1)
    return folder

### Test file processing functions

In [24]:
# test the getListOfDafnyFiles, getListOfDafnyFolders and getFolder functions
# local repo path needs to be set prior to running the tests and `if False` 
# must be changed to `if True`
if False:
    repo_directory = "/home/roberto/projects_offline/lavoro/consensys/content/eth2.0-dafny-for-stats"
    exclude_folders_rel_path = ["src/dafny/libraries/integers"]

    exclude_folders_full_path = [os.path.join(repo_directory,f) for f in exclude_folders]

    print("Test getListOfDafnyFiles: ")
    files = getListOfDafnyFiles(repo_directory, exclude_folders_full_path)
    for i in files:
        print(i)
    print("Length of returned list: ", len(files))

    print("Test getListOfDafnyFolders: ")
    directories = getListOfDafnyFolders(files)
    for i in directories:
        print(i)
    print("Length of returned list: ", len(directories))

    print("Test getFolder for each file in files: ")
    for file in files:
        print(getFolder(repo_directory, file))

## Functions to collect statistics

In [25]:
# count the number of "lemmas" in a given dafny file
# this function uses a subprocess call
# an alternative method would be to read and search the file directly
def getLemmas(dafny_file):
    cmd = "cat " + dafny_file +"| grep lemma | wc -l"
    result = subprocess.run(['/bin/bash', '-i', '-c', cmd], stdout=subprocess.PIPE)
    return result.stdout.strip().decode('ascii')

In [26]:
# count the number of "function methods" in a given dafny file
# this function uses a subprocess call
# an alternative method would be to read and search the file directly
def getFunctions(dafny_file):
    cmd = "cat " + dafny_file +"| grep function | grep method | wc -l"
    result = subprocess.run(['/bin/bash', '-i', '-c', cmd], stdout=subprocess.PIPE)
    return result.stdout.strip().decode('ascii')

In [27]:
# count the number of ghost (= function and lemmas) processes
# ignores function methods
# to be referred to as "Theorems" in the data display
def getGhost(dafny_file):
    tmp_file = open(dafny_file, "r")
    count = 0
    for line in tmp_file.readlines():
        if line.strip().startswith(("function", "lemma")):
            if not line.strip().startswith("function method"):
                count += 1
                #print(line)
    tmp_file.close()
    return count

In [28]:
# count the number of non-ghost ()= function methods and methods and predicates) processes
# to be referred to as "Implementations" in the data display
def getNonGhost(dafny_file):
    tmp_file = open(dafny_file, "r")
    count = 0
    for line in tmp_file.readlines():
        if line.strip().startswith(("function method", "method", "predicate")):
            count += 1
            #print(line)

    tmp_file.close()
    return count

In [66]:
# count the number of lines of code
# the count occurs after the dafny file is printed used the compiler
# the count also occurs after this output has been cleaned
def getLoC(dafny_file):
    show_ghost = True
    executable = "dafny"
    args  = [] 
    args += ['/rprint:-']
    args += ["/noAutoReq"]
    args += ["/noVerify"]
    # args += ["/nologo"]
    args += ["/env:0"]
    if show_ghost:
        args += ["/printMode:NoIncludes"]
    else:
        args += ["/printMode:NoGhost"]
    args += [dafny_file]
    cmd = ' '.join([executable] + args)
    result = subprocess.run(['/bin/bash', '-i', '-c', cmd], stdout=subprocess.PIPE)
    output = result.stdout.decode('ascii')
    #print(type(result.stdout.decode('ascii')))
    #print(result.stdout.decode('ascii'))

    #remove this section once code has be tested OR comment out
    #tmp_file = open("tmp.txt", "w")
    #tmp_file.write(result.stdout.decode('ascii'))
    #tmp_file.close()
    ######---------------------

    count = 0
    for line in output.splitlines():
        # clean output i.e. remove comment at start and verifier status
        if line.startswith(("Dafny program verifier did not attempt verification", "//")):
            #print(i)
            pass
        else:
            if line.strip():
                count += 1
                #print(line)
    #print("#LoC: ", count)
    return count

In [30]:
# count the number of lines included in the license comment
# assumes license comment is at the start of the file and is of format /* ... */
# assumes that it has been confirmed that the file has a license comment
def getLicenseLineCount(dafny_file):
    tmp_file = open(dafny_file, "r")
    count = 0
    flag = 0
    for line in tmp_file.readlines():
        tmp_line = line.strip()
        cleaned = ' '.join(i for i in tmp_line.split() if i not in ["//", "/*", "/**", "*", "*/"])
        if (not flag) and (tmp_line.startswith("/*")):
            if cleaned:
                count += 1
            flag = 1
        elif flag:
            if cleaned:
                count += 1
            if tmp_line.startswith("*/"):
                tmp_file.close()
                return count

In [31]:
# count the number of lines of documentation
# don't include license comment or empty comment lines
def getDocumentation(dafny_file):
    tmp_file = open(dafny_file, "r")
    count = 0
    license_flag = 0
    for line in tmp_file.readlines():
        tmp_line = line.strip()
        if tmp_line.startswith(("//", "/*", "/**", "*", "*/")):
            cleaned = ' '.join(i for i in tmp_line.split() if i not in ["//", "/*", "/**", "*", "*/"])
            if cleaned:
                #print(cleaned)
                count += 1
                #print(line)
        if tmp_line.startswith("* Copyright 2021 ConsenSys Software Inc."):
            license_flag = 1

    tmp_file.close()
    if license_flag:
        count -= getLicenseLineCount(dafny_file)
        #print(getLicenseLineCount(dafny_file))
    return count

In [83]:
# count the number of theorems (getGhost) and implementations (getNonGhost) proved
# i.e. check that the number of errors when verified is zero
# TODO: include arguments for getGhost and getNonGhost to reduce duplicate processing
def getProved(dafny_file):
    cmd = "dafny /dafnyVerify:1 /compile:0 /noCheating:1 /vcsMaxKeepGoingSplits:10 /vcsCores:12 /vcsMaxCost:1000  /vcsKeepGoingTimeout:30  /vcsFinalAssertTimeout:90  /verifySeparately " + dafny_file
    result = subprocess.run(['/bin/bash', '-i', '-c', cmd], stdout=subprocess.PIPE)
    output = result.stdout.decode('ascii')
    # print(result)
    for line in output.splitlines():
        if line.startswith("Dafny program verifier finished with "):
            # check no errors
            # print(line, re.findall(r'\d+', line)[1], type(re.findall(r'\d+', line)[1]))
            if not int(re.findall(r'\d+', line)[1]):
                return (getGhost(dafny_file) + getNonGhost(dafny_file))
        else:
            pass
    # if the verifier doesn't finish, return -1
    return 0

### Test statistics functions

In [75]:
# s/False/True if need to run the tests
if False:
    # test file options:
    test_file = "/Users/franck/development/eth2.0-dafny/src/dafny/beacon/gasper/GasperProofs.p.dfy"
    #test_file = "/Users/franck/development/eth2.0-dafny/test/dafny/merkle/Merkleise.test.dfy"
    #test_file = "/Users/franck/development/eth2.0-dafny/test/dafny/ssz/BitListSeDes.tests.dfy"
    #test_file = "/Users/franck/development/eth2.0-dafny/src/dafny/ssz/BitListSeDes.dfy"
    #test_file = "/Users/franck/development/eth2.0-dafny/src/dafny/merkle/Merkleise.dfy"

    #print("Lemmas ...")
    #print(getLemmas(test_file))

    #print("Function methods ...")
    #print(getFunctions(test_file))

    print("LoC ...")
    print(getLoC(test_file))

    #print("Documentation ...")
    #print(getDocumentation(test_file))

    print("Proved (verified from compile) ...")
    print(getProved(test_file))

    #print("Ghost ...")
    #rint(getGhost(test_file))

    #print("NonGhost ...")
    #print(getNonGhost(test_file))

LoC ...
268
Proved (verified from compile) ...
CompletedProcess(args=['/bin/bash', '-i', '-c', 'dafny /dafnyVerify:1 /compile:0 /noCheating:1 /vcsMaxKeepGoingSplits:10 /vcsCores:12 /vcsMaxCost:1000  /vcsKeepGoingTimeout:30  /vcsFinalAssertTimeout:90  /verifySeparately /Users/franck/development/eth2.0-dafny/src/dafny/beacon/gasper/GasperProofs.p.dfy'], returncode=0, stdout=b'/Users/franck/development/eth2.0-dafny/src/dafny/beacon/gasper/../../utils/SetHelpers.dfy(60,22): Warning: /!\\ No terms found to trigger on.\n/Users/franck/development/eth2.0-dafny/src/dafny/beacon/gasper/./GasperEBBs.dfy(91,16): Warning: /!\\ No terms found to trigger on.\n/Users/franck/development/eth2.0-dafny/src/dafny/beacon/gasper/./GasperEBBs.dfy(159,16): Warning: /!\\ No terms found to trigger on.\n\nDafny program verifier finished with 12 verified, 0 errors\n')
Dafny program verifier finished with 12 verified, 0 errors 0 <class 'str'>
10


## Collate results into a pandas dataframe

One row per Dafny file.

In [78]:
import tempfile

# create a pandas dataframe to store stats relating to the dafny files
column_list = ['Files', 'Folder', '#LoC', 'Theorems', 'Implementations', "Documentation", "#Doc/#LoC (%)", "Proved"]

# list here all the directory not to include in the stat collection with path relative to the root of the repo
exclude_folders_rel_path = []

# performs a clean checkout from GitHub before collecting the stats
with tempfile.TemporaryDirectory() as repo_directory: 
    # subprocess.run(['/bin/bash','-c','git clone git@github.com:ConsenSys/eth2.0-dafny.git -b goal1 --single-branch ' + repo_directory], stdout=subprocess.PIPE)

    repo_directory = "/Users/franck/development/eth2.0-dafny/src/dafny/"

    exclude_folders_full_path = [os.path.join(repo_directory,f) for f in exclude_folders_rel_path]

    files = getListOfDafnyFiles(repo_directory, exclude_folders_full_path)   

    df = pd.DataFrame(columns=column_list)

    # collect data for each dafny file
    for file in files:
        loc = getLoC(file)
        ghost = getGhost(file)
        nonghost = getNonGhost(file)
        doc = getDocumentation(file)
        proved = getProved(file)
        df2 = pd.DataFrame([[os.path.basename(file), 
                            getFolder(repo_directory, file), 
                            loc ,
                            ghost, 
                            nonghost,
                            doc,
                            round(doc/loc * 100),
                            proved]], 
                            columns=column_list)
        df = df.append(df2, ignore_index=True)

    # create and append totals for numeric columns
    totals = pd.DataFrame([["", 
                            "TOTAL", 
                            df['#LoC'].sum(),
                            df['Theorems'].sum(), 
                            df['Implementations'].sum(),
                            df['Documentation'].sum(),
                            round(df['Documentation'].sum()/df['#LoC'].sum() * 100),
                            df['Proved'].sum()]], 
                            columns=column_list)
    df = df.append(totals, ignore_index=True)

    # convert numeric columns to int64
    numCols = ['#LoC', 'Theorems', 'Implementations', "Documentation", "#Doc/#LoC (%)", "Proved"]
    df[numCols] = df[numCols].astype("int64")

    #display a sample of rows
    df.head(len(df))

CompletedProcess(args=['/bin/bash', '-i', '-c', 'dafny /dafnyVerify:1 /compile:0 /noCheating:1 /vcsMaxKeepGoingSplits:10 /vcsCores:12 /vcsMaxCost:1000  /vcsKeepGoingTimeout:30  /vcsFinalAssertTimeout:90  /verifySeparately /Users/franck/development/eth2.0-dafny/src/dafny/beacon/BeaconChainTypes.dfy'], returncode=0, stdout=b'/Users/franck/development/eth2.0-dafny/src/dafny/beacon/attestations/../../utils/SetHelpers.dfy(60,22): Warning: /!\\ No terms found to trigger on.\n\nDafny program verifier finished with 12 verified, 0 errors\n')
Dafny program verifier finished with 12 verified, 0 errors 0 <class 'str'>
CompletedProcess(args=['/bin/bash', '-i', '-c', 'dafny /dafnyVerify:1 /compile:0 /noCheating:1 /vcsMaxKeepGoingSplits:10 /vcsCores:12 /vcsMaxCost:1000  /vcsKeepGoingTimeout:30  /vcsFinalAssertTimeout:90  /verifySeparately /Users/franck/development/eth2.0-dafny/src/dafny/beacon/Helpers.dfy'], returncode=0, stdout=b'/Users/franck/development/eth2.0-dafny/src/dafny/beacon/attestations/.

### Alternative format

May be useful for github

In [82]:
from tabulate import tabulate

print(tabulate(df, headers='keys', tablefmt='github'))


|    | Files                   | Folder                 |   #LoC |   Theorems |   Implementations |   Documentation |   #Doc/#LoC (%) |   Proved |
|----|-------------------------|------------------------|--------|------------|-------------------|-----------------|-----------------|----------|
|  0 | BeaconChainTypes.dfy    | beacon                 |     43 |          0 |                 1 |             166 |             386 |        1 |
|  1 | Helpers.dfy             | beacon                 |     96 |          2 |                 8 |              35 |              36 |       10 |
|  2 | AttestationsHelpers.dfy | beacon/attestations    |    255 |          9 |                11 |             159 |              62 |       20 |
|  3 | AttestationsTypes.dfy   | beacon/attestations    |     36 |          0 |                 1 |              89 |             247 |        1 |
|  4 | ForkChoice.dfy          | beacon/forkchoice      |    238 |          3 |                15 |             176 | 

## Group data

One row per folder.

In [89]:
# create a pandas dataframe to store stats relating to the dafny files
# stats grouped by folder
column_list = ['Folder', '#Files', '#LoC', 'Theorems', 'Implementations', "Documentation", "#Doc/#LoC (%)", "Proved"]
df_grouped = pd.DataFrame(columns=column_list)
exclude_folders_rel_path = ["ssz", "utils"]

with tempfile.TemporaryDirectory() as repo_directory:
    # subprocess.run(['/bin/bash','-c','git clone git@github.com:ConsenSys/eth2.0-dafny.git ' + repo_directory], stdout=subprocess.PIPE)

    repo_directory = "/Users/franck/development/eth2.0-dafny/src/dafny/"
    exclude_folders_full_path = [os.path.join(repo_directory,f) for f in exclude_folders_rel_path]

    # TODO: We currently get the list of folders out of the list of files and then in the `for` loop
    # we retrieve the list of files again for each folder. We may want to think of a more elegant 
    # implementation.
    allFiles = getListOfDafnyFiles(repo_directory, exclude_folders_full_path)  

    folders = getListOfDafnyFolders(allFiles)

    for folder in folders:
        files = getListOfDafnyFiles(folder)
        
        nFiles = 0
        nLoc = 0
        nGhost = 0
        nNonGhost = 0
        nDoc = 0
        nProved = 0
        for file in files:
            nFiles += 1
            nLoc += getLoC(file)
            nGhost += getGhost(file)
            nNonGhost += getNonGhost(file)
            nDoc += getDocumentation(file)
            nProved += getProved(file)
        

        df2 = pd.DataFrame([[getFolder(repo_directory, file), 
                            nFiles, 
                            nLoc ,
                            nGhost, 
                            nNonGhost,
                            nDoc,
                            round(nDoc/nLoc * 100),
                            nProved]], 
                            columns=column_list)
        df_grouped = df_grouped.append(df2, ignore_index=True)

    #display a sample of rows
    df_grouped.head(len(df_grouped))


CompletedProcess(args=['/bin/bash', '-i', '-c', 'dafny /dafnyVerify:1 /compile:0 /noCheating:1 /vcsMaxKeepGoingSplits:10 /vcsCores:12 /vcsMaxCost:1000  /vcsKeepGoingTimeout:30  /vcsFinalAssertTimeout:90  /verifySeparately /Users/franck/development/eth2.0-dafny/src/dafny/beacon/BeaconChainTypes.dfy'], returncode=0, stdout=b'/Users/franck/development/eth2.0-dafny/src/dafny/beacon/attestations/../../utils/SetHelpers.dfy(60,22): Warning: /!\\ No terms found to trigger on.\n\nDafny program verifier finished with 12 verified, 0 errors\n')
CompletedProcess(args=['/bin/bash', '-i', '-c', 'dafny /dafnyVerify:1 /compile:0 /noCheating:1 /vcsMaxKeepGoingSplits:10 /vcsCores:12 /vcsMaxCost:1000  /vcsKeepGoingTimeout:30  /vcsFinalAssertTimeout:90  /verifySeparately /Users/franck/development/eth2.0-dafny/src/dafny/beacon/Helpers.dfy'], returncode=0, stdout=b'/Users/franck/development/eth2.0-dafny/src/dafny/beacon/attestations/../../utils/SetHelpers.dfy(60,22): Warning: /!\\ No terms found to trigger o

### Print dataframe to .csv, .tex and .pdf

In [88]:
# create filenames that include the current data string
timestr = time.strftime("%Y-%m-%d-%H:%M")
rawfile = 'data' + timestr + '.csv'
grouped_rawfile = 'dataGrouped' + timestr + '.csv'
filename = 'data' + timestr + '.tex'
pdffile = 'data' + timestr + '.pdf'

# check if data directory already exists and create if necessary
if not os.path.exists('data'):
    os.makedirs('data')

#print to csv file without an index
df.to_csv("data/" + rawfile, index = False)
df_grouped.to_csv("data/" + grouped_rawfile, index = False)

#print to pdf via latex
template = r'''\documentclass[a4paper, 12pt]{{article}}
\usepackage[landscape]{{geometry}}
\usepackage{{booktabs}}
\begin{{document}}
\section*{{https://github.com/ConsenSys/eth2.0-dafny}}
\subsection*{{Data collected: {}}}
\scriptsize
{}
\vspace{{2em}}
{}
\end{{document}}
'''

with open(filename, 'w') as f:
    f.write(template.format(time.strftime("%Y-%m-%d-%H:%M"), df.to_latex(index=False), df_grouped.to_latex(index=True)))

subprocess.call(['pdflatex', filename])

# remove surplus files and move .csv, .tex and .pdf files to the data folder
os.remove('data' + timestr + '.log')
os.remove('data' + timestr + '.aux')

shutil.move(filename, "data/" + filename)
shutil.move(pdffile, "data/" + pdffile)

'data/data2021-08-03-07:52.pdf'